In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os

import descarteslabs as dl
import geopandas as gpd
from tqdm.notebook import tqdm

from scripts import deploy_candidate_detect, candidate_detect

In [ ]:
roi = 'brazil'
model_name = '0.0.11'
product_name = f'{roi}_v{model_name}_2019-01-01_2021-06-01'
pred_threshold = 0.6
min_sigma = 3.5
product_id = f'earthrise:{f"{product_name}_blobs_thresh_{pred_threshold}_min-sigma_{min_sigma}_area-thresh_0.0025"}'

run_local = False

args = [
    '--product_name',
    product_name,
    '--pred_threshold',
    str(pred_threshold),
    '--min_sigma',
    str(min_sigma),
]
if run_local:
    args.append('--run_local')

In [ ]:
# Because of the way DL uploads modules when queuing async tasks, we need to launch from the scripts/ folder
%cd ../scripts
%pwd

In [ ]:
deploy_candidate_detect.main(args)

### Download data after the task is complete

In [ ]:
fc_id = [fc.id for fc in dl.vectors.FeatureCollection.list() if product_id in fc.id][0]
fc = dl.vectors.FeatureCollection(fc_id)

roi_file = f'../data/boundaries/{roi}.geojson'
region = gpd.read_file(roi_file)['geometry']

features = []
for elem in tqdm(fc.filter(region).features()):
    features.append(elem.geojson)

In [ ]:
coords = [feat['geometry']['coordinates'] for feat in features]
merged = candidate_detect.merge_similar_sites(coords, search_radius=0.01, plot=True)
directory = f"../data/model_outputs/candidate_sites/{model_name}"
merged.to_file(os.path.join(directory, product_id.split(':')[-1] + '.geojson'))